In [19]:
import json
import pennylane as qml
from pennylane import numpy as np

In [21]:
# Uneditable section #
dev = qml.device("default.qubit", wires=["e1", "e2", "e3", "e4", "result"], shots=1)
#dev.operations.add("op")

wires = ["e1", "e2", "e3", "e4", "result"]
# End of uneditable section #

@qml.qnode(dev)
def circuit(project_execution):
    """This is the circuit we will use to detect which is the lazy worker. Remember
    that we will only execute one shot.

    Args:
        project_execution (qml.ops):
            The gate in charge of marking in the last qubit if the project has been finished
            as indicated in the statement.

    Returns:
        (numpy.tensor): Measurement output in the 5 qubits after a shot.
    """

    ## This is Grover search over the reduced 4-dimensional subspace
    # init |0111>|0> + |1011>|0> + |1101>|0> + |1110>|0>
    n = len(wires) - 1
    for i in range(n): # prep i-th qubit
        if i == 0: # non-controlled RX
            qml.RX(np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )
        qml.PauliX(wires=wires[i]) # rotate i-th qubit to 1

    # phase flip the solution
    # e.g. -|0111>|-> + |1011>|-> + |1101>|-> + |1110>|->
    qml.PauliX(wires="result")
    qml.Hadamard(wires="result")
    project_execution(wires=wires)

    # reverse the initialisation
    for i in range(n)[::-1]: # prep i-th qubit
        qml.PauliX(wires=wires[i])
        if i == 0: # non-controlled RX
            qml.RX(-np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                -np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )

    # phase flip all the initial state |0000>
    for i in wires[:-1]:
        qml.PauliX(wires=i)
    qml.ctrl(qml.PauliZ, control=wires[:-2])(wires=wires[-2])
    for i in wires[:-1]:
        qml.PauliX(wires=i)

    # redo the initialisation
    for i in range(n): # prep i-th qubit
        if i == 0: # non-controlled RX
            qml.RX(np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )
        qml.PauliX(wires=wires[i])

    return qml.sample(wires=dev.wires)

def process_output(measurement):
    """This function will take the circuit measurement and process it to determine who is the lazy worker.

    Args:
        measurement (numpy.tensor): Measurement output in the 5 qubits after a shot.

    Returns:
        (str): This function must return "e1", "e2" "e3" or "e4" - the lazy worker.
    """
    # Identify the lazy employee (qubit with 0 in measurement)
    for i, employee in enumerate(["e1", "e2", "e3", "e4"]):
        if measurement[i] == 0:
            return employee
    return "e1"  # Default return (should not occur with correct circuit)

In [41]:
def run(test_case_input: str) -> str:
    return None

def check(t: int) -> None:
    samples = t

    solutions = []
    output = []

    for s in range(samples):
        lazy = np.random.randint(0, 4)
        no_lazy = list(range(4))
        no_lazy.pop(lazy)

        def project_execution(wires):
            class op(qml.operation.Operator):
                num_wires = 5

                def compute_decomposition(self, wires):
                    raise ValueError("You cant descompose this gate")

                def matrix(self):
                    m = np.zeros([32, 32])
                    for i in range(32):
                        b = [int(j) for j in bin(64 + i)[-5:]]
                        if sum(np.array(b)[no_lazy]) == 3:
                            if b[-1] == 0:
                                m[i, i + 1] = 1
                            else:
                                m[i, i - 1] = 1
                        else:
                            m[i, i] = 1
                    return m

            op(wires=wires)
            return None

        out = circuit(project_execution)
        solutions.append(lazy + 1)
        output.append(int(process_output(out)[-1]))

    print(f"{solutions}\n{output}")

    assert np.allclose(
        output, solutions, rtol=1e-4
    ), "Your circuit does not give the correct output."

    ops = [op.name for op in circuit.tape.operations]
    assert ops.count("op") == 1, "You have used the oracle more than one time."

In [43]:
check(500)

[3, 3, 3, 3, 1, 2, 2, 4, 2, 3, 2, 1, 3, 1, 4, 2, 2, 2, 2, 4, 3, 3, 1, 1, 4, 3, 4, 1, 1, 4, 1, 4, 4, 3, 1, 2, 3, 3, 1, 2, 4, 2, 1, 1, 3, 2, 3, 4, 2, 1, 2, 1, 4, 3, 4, 1, 2, 2, 3, 4, 2, 4, 3, 1, 2, 2, 3, 2, 4, 1, 3, 4, 4, 3, 1, 1, 4, 1, 4, 2, 1, 2, 2, 3, 3, 3, 4, 1, 4, 2, 3, 3, 2, 4, 3, 2, 1, 3, 1, 2, 4, 4, 4, 2, 4, 4, 4, 1, 2, 4, 3, 3, 1, 4, 2, 2, 2, 3, 3, 3, 4, 3, 2, 4, 1, 3, 4, 4, 1, 3, 3, 3, 1, 4, 1, 4, 3, 2, 2, 3, 1, 4, 3, 4, 1, 1, 2, 4, 1, 2, 4, 3, 1, 1, 3, 1, 2, 2, 2, 3, 4, 4, 4, 1, 2, 1, 2, 4, 4, 3, 3, 4, 1, 1, 1, 4, 3, 1, 3, 1, 1, 2, 4, 3, 1, 4, 3, 3, 4, 4, 4, 1, 3, 2, 2, 2, 3, 3, 1, 1, 4, 2, 4, 1, 3, 1, 3, 1, 2, 3, 1, 1, 2, 2, 2, 4, 4, 3, 2, 4, 3, 2, 2, 3, 3, 1, 1, 2, 2, 1, 3, 4, 1, 4, 4, 2, 4, 4, 2, 3, 4, 3, 4, 4, 2, 1, 2, 1, 2, 3, 4, 2, 1, 4, 4, 3, 3, 4, 1, 2, 3, 3, 3, 1, 1, 3, 1, 3, 2, 1, 3, 4, 3, 4, 2, 4, 3, 3, 1, 2, 3, 3, 1, 1, 1, 2, 3, 2, 4, 1, 3, 2, 1, 3, 4, 2, 3, 1, 4, 2, 2, 1, 3, 4, 4, 3, 4, 3, 4, 1, 2, 4, 1, 2, 1, 1, 2, 1, 1, 2, 4, 3, 2, 1, 2, 2, 3, 2, 1, 4, 4, 1, 4, 